In [1]:
from railjson_generator import (
    InfraBuilder,
    SimulationBuilder,
    Location,
)
from railjson_generator.schema.infra.direction import Direction

from  railjson_generator.schema.infra.infra import Infra


In [2]:
builder = InfraBuilder()


In [3]:
T = [
    builder.add_track_section(label='T'+str(id), length=500)
    for id in range(6)
]

In [4]:
builder.add_link(T[2].end(), T[3].begin())
builder.add_point_switch(
    T[2].begin(),
    T[0].end(),
    T[1].end(),
    label='CVG',
)
builder.add_point_switch(
    T[3].end(),
    T[4].begin(),
    T[5].begin(),
    label='DVG',
)

PointSwitch(label='DVG', delay=0, base=TrackEndpoint(track_section=TrackSection(length=500, label='T3', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.END: 1>), left=TrackEndpoint(track_section=TrackSection(length=500, label='T4', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.BEGIN: 0>), right=TrackEndpoint(track_section=TrackSection(length=500, label='T5', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.BEGIN: 0>))

In [5]:
D = [
    T[i].add_detector(label='D'+str(i), position=450)
    if i in [0, 1, 3]
    else T[i].add_detector(label='D'+str(i), position=50)
    for i in range(6)
]

In [6]:
D

[Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D0'),
 Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D1'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D2'),
 Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D3'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D4'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D5')]

In [7]:
S = [
    T[i].add_signal(D[i].position-20, Direction.START_TO_STOP, D[i])
    for i, _ in enumerate(D)
]
LS = [
    signal.add_logical_signal("BAL", settings={"Nf": "true"})
    for signal in S
]

In [8]:
LS

[LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'})]

In [9]:
stations = [
    builder.add_operational_point(label='station'+str(i))
    for i in range(2)
]
for track in range(2):
    stations[0].add_part(T[track], 300)
for track in range(4,6):
    stations[1].add_part(T[track], 300)

In [10]:
built_infra = builder.build()
infra = built_infra.to_rjs().dict()

In [11]:
{
        station['id']: len(station['parts'])
        for station in infra['operational_points']
    }

{'station0': 2, 'station1': 2}

In [15]:
built_infra

Infra(track_sections=[TrackSection(length=500, label='T0', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[BufferStop(position=0, label='buffer_stop.0'), Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D0')], signals=[Signal(position=430, direction=<Direction.START_TO_STOP: 0>, linked_detector=Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D0'), sight_distance=400, logical_signals=[LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'})], label='signal.0', installation_type='CARRE', side=<Side.LEFT: 'LEFT'>, angle=90)], operational_points=[OperationalPointPart(operational_point=OperationalPoint(label='station0', trigram='STA', parts=[..., OperationalPointPart(operational_point=..., position=300)]), position=300)], coordinates=[(0, 0), (0, 0)], slopes=[], curves=[]), TrackSection(length=500, label='T1', track_name='placeholder_track

In [21]:
sim_builder = SimulationBuilder(built_infra)

for train_id in range(2):
    sim_builder.add_train_schedule(
        Location(built_infra.track_sections[train_id], 300),
        Location(built_infra.track_sections[train_id+3], 300),
        label='train'+str(train_id),
        departure_time=0
    )
simulation = sim_builder.build().format()

In [22]:
len(simulation['train_schedules'])

2

In [ ]:
import rlway.osrd.infra as infr

In [ ]:
infr.swi(infra)

['rt.D0->D2',
 'rt.buffer_stop.0->D0',
 'rt.D1->D2',
 'rt.buffer_stop.1->D1',
 'rt.D2->D3',
 'rt.D3->D5',
 'rt.D3->D4',
 'rt.D4->buffer_stop.2',
 'rt.D5->buffer_stop.3']

In [ ]:
infr.route_lengths(infra)

{'rt.D0->D2': 100.0,
 'rt.buffer_stop.0->D0': 450.0,
 'rt.D1->D2': 100.0,
 'rt.buffer_stop.1->D1': 450.0,
 'rt.D2->D3': 900.0,
 'rt.D3->D5': 100.0,
 'rt.D3->D4': 100.0,
 'rt.D4->buffer_stop.2': 450.0,
 'rt.D5->buffer_stop.3': 450.0}

In [ ]:
{"a": 1, "b": 2} == {"b": 2, "a": 1, }

True

In [ ]:
print(infr.draw_infra(infra))

None


In [ ]:
infra['switches']

[{'id': 'CVG',
  'switch_type': 'point',
  'group_change_delay': 0.0,
  'ports': {'base': {'endpoint': <Endpoint.BEGIN: 'BEGIN'>, 'track': 'T2'},
   'left': {'endpoint': <Endpoint.END: 'END'>, 'track': 'T0'},
   'right': {'endpoint': <Endpoint.END: 'END'>, 'track': 'T1'}},
  'extensions': {'sncf': {'label': 'CVG'}}},
 {'id': 'DVG',
  'switch_type': 'point',
  'group_change_delay': 0.0,
  'ports': {'base': {'endpoint': <Endpoint.END: 'END'>, 'track': 'T3'},
   'left': {'endpoint': <Endpoint.BEGIN: 'BEGIN'>, 'track': 'T4'},
   'right': {'endpoint': <Endpoint.BEGIN: 'BEGIN'>, 'track': 'T5'}},
  'extensions': {'sncf': {'label': 'DVG'}}}]

In [ ]:
infra['operational_points']

[]